In [37]:
using TimerOutputs

include("CayleyVerify.jl")
include("DeepPoly.jl")

function dorefa_to_staircase(k::Int)
    n = 2^k - 1
    slopes = zeros(n+1)
    breakpoints = [-Inf]
    for i in 1:n
        push!(breakpoints, (2*i-1)/n - 1)
    end
    push!(breakpoints, Inf)
    
    constant_terms = [-1.0]
    for i in 1:n
        push!(constant_terms, -1.0 + 2*i/n)
    end
    return StaircaseFunction(breakpoints, slopes, constant_terms)
end

function predict(neural_net, img)
    num_layers = length(neural_net.weights)
    a = img'
    for i in 1:num_layers
        a = a * neural_net.weights[i] + neural_net.biases[i]'
        if i <= num_layers -1
            a = [eval(neural_net.activation[i], a[j]) for j in 1:length(a)]'
        end
    end
    output = a'
    return findmax(output)[2]
end

predict (generic function with 1 method)

In [65]:
using Pickle
using Suppressor
using Printf

net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa3_Dense256-Dense256.pkl"))
f = dorefa_to_staircase(3)
activation = [f, f]
neural_net = NeuralNetwork(net_from_pickle, activation)
print("net loaded")

MethodError: MethodError: Cannot `convert` an object of type Vector{Any} to an object of type Float64

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number
   @ Base twiceprecision.jl:273
  ...


In [39]:
raw_imgs = Pickle.load(open("./imgs/MNIST_images-for-verification"))
imgs = []
for img in raw_imgs
    img = vcat([w' for w in img] ...)
    img = vcat(img'...)
    push!(imgs, img)
end
labels = Pickle.load(open("./imgs/MNIST_labels-for-verification", "r+"))
labels = Array{Int64}(labels.args[2][5])
labels = [l+1 for l in labels]
print("images loaded")

# Cayley Embedding Solution

In [40]:
upper_bound = 150
lower_bound = 0
count = 1
for (img, label) in zip(imgs, labels)
    @printf("Verify %d-th image \n", count)
    vulnerable = false
    for target_label in 1:10
        if target_label != label
            @suppress begin
                opt_val, opt_sol_x, opt_sol_z = target_attack(neural_net, img, label, target_label, 0.008)
                if opt_val > 0
                    vulnerable = true
                    adv_img = [opt_sol_x[1, j] for j in 1:784]
                    pred = predict(neural_net, adv_img)
                    if pred != label
                        upper_bound -= 1
                        break
                    end
                end
            end
        end
    end
    if vulnerable == false
        lower_bound += 1
    end
    count += 1
end

UndefVarError: UndefVarError: `neural_net` not defined

In [41]:
lower_bound

0

In [42]:
upper_bound

150

In [43]:
lower_bound

0

In [44]:
upper_bound

150

In [45]:
lower_bound

0

In [46]:
upper_bound

150

In [47]:
lower_bound

0

In [48]:
upper_bound

150

In [49]:
lower_bound

0

In [50]:
upper_bound

150

In [51]:
lower_bound

0

In [52]:
upper_bound

150

In [53]:
net_from_pickle = Pickle.load(open("./models/MNIST-DoReFa2_Dense256-Dense256.pkl"))
f = dorefa_to_staircase(2)
activation = [f, f]
neural_net = NeuralNetwork(net_from_pickle, activation)
print("net loaded")

MethodError: MethodError: no method matching length(::Pickle.Defer)

Closest candidates are:
  length(!Matched::Union{Base.KeySet, Base.ValueIterator})
   @ Base abstractdict.jl:58
  length(!Matched::Union{Adjoint{T, S}, Transpose{T, S}} where {T, S})
   @ LinearAlgebra /opt/homebrew/Cellar/julia/1.9.4/share/julia/stdlib/v1.9/LinearAlgebra/src/adjtrans.jl:295
  length(!Matched::Union{SparseArrays.FixedSparseVector{Tv, Ti}, SparseArrays.SparseVector{Tv, Ti}} where {Tv, Ti})
   @ SparseArrays /opt/homebrew/Cellar/julia/1.9.4/share/julia/stdlib/v1.9/SparseArrays/src/sparsevector.jl:95
  ...


# Big-M Solution

In [54]:
upper_bound = 150
lower_bound = 0
count = 1
for (img, label) in zip(imgs, labels)
    @printf("Verify %d-th image \n", count)
    vulnerable = false
    for target_label in 1:10
        if target_label != label
            @suppress begin
                mip, _, _ = init_mip_deeppoly(neural_net, img, 0.008)
                last_layer = last(neural_net.weights)
                objective = zeros(10) # always 10 classes
                objective[target_label] = 1.0
                objective[label] = -1.0
                c = last_layer * objective

                num_layers = length(neural_net.weights)
                final_dim, output_dim = size(last_layer)
                @objective(mip, Max, sum(c[i]*mip[:x][num_layers, i] for i in 1:final_dim))
                optimize!(mip)
                opt_val = objective_value(mip)
                if opt_val > 0
                    vulnerable = true
                    break
                end
            end
        end
    end
    if vulnerable == false
        lower_bound += 1
    end
    count += 1
end

UndefVarError: UndefVarError: `neural_net` not defined

In [55]:
lower_bound

0

In [56]:
lower_bound

0

In [57]:
lower_bound

0

In [58]:
lower_bound

0

In [59]:
lower_bound

0

In [60]:
lower_bound

0